# Details 

---
**Submitted By** : Aravind D. Chakravarti  
**Batch**               : 08  
**References**     : Rohan Shravan class notes/EIP sessions  and Andrew Ng Deep Learning AI (for RESNET-50 base code; However many things are modified to meet the Tinyimagenet dataset)  
**Thanks**           : Rohan Shravan and EIP community 
                          

---

# Accuracy information
I have got about 45.68% accuracy in the 39th Epoch


---



# Other details
  * To begin with I took RESNET-50 architecture. I removed some *deep layers* of it it because we are deeling with 200 classes (not 1000 classes) and may be that much deep architecture will be *too much* for our application.
  * I increased number of channel in each layer gradually, and took it upto 2048 kernels in the last layer. -- *Why? Because we have 200 classes*
  * Took base model with Conv2D modified architecture --  Kept aim of meeting the validation accuracy, without much worrying about number of parameters. 
  * After reaching the validation accuracy, reduced the number of parameters by using "SeperableConv2D"
  * Current number of parameters: Nearly 7 million
  * Best accuracy : 45.68%
                 
                           

---

# What I understood?
  * I could able to understand *Why ResNet* and *Its advantages*
  * I could able to understand the ResNet paper by He *et al*
  * I could able to use GAP effectively


# What I missed?
  * I could not do much of the augmentation which was the best method to get this model much more validation accuracy preventing the overfitting
  
 

In [0]:
# Check the directory if we arleady have the tiny-imagenet-200 dataset downloaded
!ls

sample_data


In [0]:
## Download the dataset and unzip it
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip

--2019-04-01 17:22:28--  http://cs231n.stanford.edu/tiny-imagenet-200.zip
Resolving cs231n.stanford.edu (cs231n.stanford.edu)... 171.64.68.10
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248100043 (237M) [application/zip]
Saving to: ‘tiny-imagenet-200.zip’

tiny-imagenet-200.z 100%[===================>] 236.61M  38.8MB/s    in 7.5s    

2019-04-01 17:22:35 (31.7 MB/s) - ‘tiny-imagenet-200.zip’ saved [248100043/248100043]



In [0]:
## What if I need to unzip the data? Use below commands
# Wonder what is that -qq parameter? That is telling the Linux to be quiter when it is unzipping
!ls
!unzip -qq 'tiny-imagenet-200.zip'
!ls

sample_data  tiny-imagenet-200.zip
sample_data  tiny-imagenet-200	tiny-imagenet-200.zip


# Few basic libraries required

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib  inline
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


## Libraries for our RESNET

In [0]:
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling2D, SeparableConv2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
#from resnets_utils import *
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

In [0]:
val_data = pd.read_csv('./tiny-imagenet-200/val/val_annotations.txt', sep='\t', header=None, names=['File', 'Class', 'X', 'Y', 'H', 'W'])
val_data.drop(['X', 'Y', 'H', 'W'], axis=1, inplace=True)
val_data.head(3)

,File,Class
0,val_0.JPEG,n03444034
1,val_1.JPEG,n04067472
2,val_2.JPEG,n04070727


In [0]:
# Zoom range is kept to 50%, this should give an effect of random image resizing upto 32x32 automatically
# Some kind of skewing is done using width shift and height shift
# 60 degree roation is done
train_datagen = ImageDataGenerator(
    rescale= 1./255,
    zoom_range = 0.5,
    width_shift_range=0.4,
    height_shift_range=0.4,
    rotation_range=60,
    horizontal_flip=True
    )

valid_datagen = ImageDataGenerator(rescale=1./255)

In [0]:
# Get the training data, using batch size of 512
train_generator = train_datagen.flow_from_directory( r'./tiny-imagenet-200/train/', target_size=(64, 64), color_mode='rgb', 
                                                    batch_size=512, class_mode='categorical', shuffle=True, seed=42)

Found 100000 images belonging to 200 classes.


In [0]:
# Validation data
validation_generator = valid_datagen.flow_from_dataframe(val_data, directory='./tiny-imagenet-200/val/images/', x_col='File', y_col='Class', target_size=(64, 64),
                                                    color_mode='rgb', class_mode='categorical', batch_size=512, shuffle=True, seed=42)

Found 10000 images belonging to 200 classes.


## Residual network architecture is built here.. 
### There are two kinds of blocks 'identity_block' and 'convolution block'

  * Identity block is used when the input dimension is same as that of output block
  * Convolution block is used when the input dimension is **not** same as that of output block


In [0]:
def identity_block(X, f, filters, stage, block):
    """
    X       - Input batch of images 
    filters - What should be the filter dimensions 
    stage   - Just for naming convensions
    block   - Just for naming convensions
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Get the filters
    F1, F2, F3 = filters
    
    # Saving this for shortcut connections
    X_shortcut = X
    
    # We have 
    # Input --> Conv  --> Conv --> Conv with addition
    
    X = SeparableConv2D(filters = F1, kernel_size = (3, 3), strides = (1,1), padding = 'same', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
      
    X = SeparableConv2D(filters=F2, kernel_size=(f, f), strides=(1,1), padding='same', name = conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    
    X = SeparableConv2D(filters=F3, kernel_size=(3,3), strides=(1,1), padding='same', name = conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name = bn_name_base + '2c')(X)
    
    X = Add()([X, X_shortcut])
      
    X = Activation('relu')(X)
    
    
    return X

In [0]:
def convolutional_block(X, f, filters, stage, block, s = 1):
    """
    Arguments:
    X       - Input batch of images 
    f       - filter shape
    filters - What should be the filter dimensions 
    stage   - Just for naming convensions
    block   - Just for naming convensions
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # For shortcut
    X_shortcut = X


    # First conv stage
    X = SeparableConv2D(F1, (3, 3), strides = (s,s), padding= 'same', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    # 2nd 
    X = SeparableConv2D(F2, (f, f), strides = (1,1), padding= 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # 3rd
    X = SeparableConv2D(F3, (3, 3), strides = (1,1), padding='same', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name = bn_name_base + '2c')(X)

    # Shortcut
    X_shortcut = SeparableConv2D(F3, (3,3), strides=(s,s), padding='same', name = conv_name_base + '1', kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name = bn_name_base + '1')(X_shortcut)

    # Merge
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
      
    return X

## Lets go ahead and build Resnet-50 (With some modifications)

In [0]:
def ResNet50(input_shape = (64, 64, 3), classes = 200):
    """
    input_shape : shape of the images of the dataset
    classes     : integer, number of classes

    """
    
    X_input = Input(input_shape)

    # I know next line is not necessary but kept it for debugging purpose
    X = X_input
    
    # Let us build some conv layer - These will probably get us the edges and gradients in image
    X = Conv2D(64, (3, 3), strides = (1, 1), padding = 'same', name = 'res1a_conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'pre1a_bn_conv1')(X)
    X = Activation('relu')(X)
    
    # Reduce the size of the image
    X = MaxPooling2D((2, 2), strides=(2, 2))(X)   #32
        
    # Do some more convolutions
    X = convolutional_block(X, f = 3, filters = [32, 64, 128], stage = 2, block='a')
    
    # Reduce the size of the image
    X = MaxPooling2D((2, 2), strides=(2, 2))(X)   #16
   
    # Convolutions
    X = convolutional_block(X, f=3, filters=[64, 128, 256], stage = 3, block='a')
    X = identity_block(X, f=3, filters=[64, 128, 256], stage = 3, block='b')
    
    # Reduce the size of the image
    X = MaxPooling2D((2, 2), strides=(2, 2))(X)   #8
    
    # Convoulutions
    X = convolutional_block(X, f=3, filters=[256,512,2048], stage = 4, block = 'a')
    X = identity_block(X, f=3, filters=[512,1024,2048], stage = 4, block = 'b')
    
    # Last convolutions with number of channels = number of classes
    X = SeparableConv2D(200, (3, 3), strides = (1, 1), padding = 'same', name = 'last_conv', kernel_initializer = glorot_uniform(seed=0))(X)
 
    # GAP layer
    X = GlobalAveragePooling2D(data_format='channels_last')(X)
   
    # Softmax  
    X = Activation('softmax')(X)
    
    
    # Create model. Note: It is not exactly ResNet50 but kind of ResNET 50 .. Hence keeping it same
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [0]:
model = ResNet50(input_shape = (64, 64, 3), classes = 200)

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
res1a_conv1 (Conv2D)            (None, 64, 64, 64)   1792        input_1[0][0]                    
__________________________________________________________________________________________________
pre1a_bn_conv1 (BatchNormalizat (None, 64, 64, 64)   256         res1a_conv1[0][0]                
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 64, 64, 64)   0           pre1a_bn_conv1[0][0]             
__________________________________________________________________________________________________
max_poolin

In [0]:
from keras.callbacks import EarlyStopping,ModelCheckpoint
import os

checkpoint = ModelCheckpoint('weights.{epoch:04d}-{val_loss:.2f}.hdf5',
                             monitor='val_loss', 
                             save_best_only=False,
                             save_weights_only = True,
                             period = 5)

callbacks_list = [checkpoint]

In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
model.fit_generator(train_generator, epochs=50, steps_per_epoch=200, validation_steps=200, validation_data=validation_generator, callbacks = callbacks_list)

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
200/200 [==============================] - 447s 2s/step - loss: 4.6842 - acc: 0.0569 - val_loss: 4.2373 - val_acc: 0.1080
Epoch 2/50
200/200 [==============================] - 420s 2s/step - loss: 4.2352 - acc: 0.1134 - val_loss: 3.8554 - val_acc: 0.1589
Epoch 3/50
200/200 [==============================] - 420s 2s/step - loss: 3.9915 - acc: 0.1459 - val_loss: 3.6466 - val_acc: 0.1975
Epoch 4/50
200/200 [==============================] - 420s 2s/step - loss: 3.7886 - acc: 0.1764 - val_loss: 3.4810 - val_acc: 0.2242
Epoch 5/50
200/200 [==============================] - 421s 2s/step - loss: 3.6283 - acc: 0.2020 - val_loss: 3.3005 - val_acc: 0.2501
Epoch 6/50
200/200 [==============================] - 421s 2s/step - loss: 3.4957 - acc: 0.2249 - val_loss: 3.1595 - val_acc: 0.2792
Epoch 7/50
200/200 [==============================] - 420s 2s/step - loss: 3.3730 - acc: 0.2439 - val_loss: 3.0562 - val_acc: 0.2999
Epoch 8/50
200/200 [=

In [0]:
!du

In [0]:
model.save_weights("Tiny_Imagenet_Basic_model2.h5")
print("Saved the model to disk")
from google.colab import files

files.download('Tiny_Imagenet_Basic_model2.h5')